In [1]:
from IPython.display import DisplayObject
import base64

class ExtensibleModule(DisplayObject):
    _MIME_TYPE = 'application/vnd.jupyter.extensible.alpha+json'
    
    def __init__(self, module=None, url=None):
        if module and url:
            raise ValueError('Cannot specify both module and url')
        if not module and not url:
            raise ValueError('Must specify either module or url')
        if module:
            url = 'data:text/javascript;base64,' + base64.b64encode(module.encode('utf-8')).decode('utf-8') 
        self._url = url
        self.data = {}
    
    def _repr_mimebundle_(self, include=None, exclude=None):
        mime_bundle = {} 
        mime_bundle[ExtensibleModule._MIME_TYPE] = {'url': self._url}
        for key in self.data.keys():
            mime_bundle[key] = self.data[key]
        return mime_bundle
        
    def _repr_javascript_module_(self):
        return {'url': 'something'}

## Can render basic content

In [2]:
ExtensibleModule(module='''
export function renderFunction(args) {
    const div = document.createElement('div');
    div.style.background = 'lime';
    div.textContent = 'passed';
    args.node.appendChild(div);
}
''')

<__main__.ExtensibleModule object>

## Can use `waitUntil` to pause processing of subsequent outputs

In [3]:
from IPython.display import Javascript
display(Javascript('window.nextOutputRendered = false'))
display(ExtensibleModule(module='''
export async function renderFunction(options) {
    const promise = new Promise((resolve) => setTimeout(resolve, 1000));
    options.data.waitUntil(promise);
    await promise;
    if (window.nextOutputRendered) {
        const div = document.createElement('div');
        div.style.background = 'red';
        div.textContent = 'failed: expected subsequent outputs to wait for this.';
        options.node.appendChild(div);
    } else {
        const div = document.createElement('div');
        div.style.background = '#88FF88';
        div.textContent = 'passed';
        options.node.appendChild(div);
    }
}
'''))
display(Javascript('window.nextOutputRendered = true'))

<IPython.core.display.Javascript object>

<__main__.ExtensibleModule object>

<IPython.core.display.Javascript object>

## Can access data of different mime types

In [36]:
data = ExtensibleModule(module='''
export function renderFunction(options) {
    const plainText = options.data.data['text/plain'];
    if (plainText != 'the text value') {
        const div = document.createElement('div');
        div.style.background = 'red';
        div.textContent = `Expected the text to be "the text value" but was ${plainText}`;
        options.node.appendChild(div);
    } else {
        const div = document.createElement('div');
        div.style.background = 'lime';
        div.textContent = 'passed';
        options.node.appendChild(div);
    }
}
''')
data.data['text/plain'] = 'the text value'
data

the text value

## Does not leak implementation details

In [5]:
ExtensibleModule(module='''
export function renderFunction(options) {
    const keys = Object.keys(options).sort();
    function fail(reason) {
        const div = document.createElement('div');
        div.style.background = 'red';
        div.textContent = reason;
        options.node.appendChild(div);
        throw new Error('failed');
    }
    
    function assertOnlyHasKeys(object, expected) {
        const actual = Object.keys(object).sort();
        expected = expected.sort();
        if (actual.length != expected.length) {
            fail(`Expected arrays to be equal: [${actual.join(',')}], [${expected.join(',')}]`);
        }
        for (let i = 0; i < actual.length; ++i) {
            if (actual[i] != expected[i]) {
                fail(`Expected arrays to be equal: [${actual.join(',')}], [${expected.join(',')}]`);
            }
        }
    }
    assertOnlyHasKeys(options, ['createComm', 'data', 'dataUpdates', 'node', 'nodeUpdates']);
    assertOnlyHasKeys(options.data, ['data', 'metadata', 'waitUntil']);
    const div = document.createElement('div');
    div.style.background = '#88FF88';
    div.textContent = 'passed';
    options.node.appendChild(div);
}
''')

<__main__.ExtensibleModule object>

In [4]:
import uuid

comm_opened = False
def my_comm_open(comm, message):
    comm.send({'passed': True})

get_ipython().kernel.comm_manager.register_target('my_comm_channel', my_comm_open)

module = ExtensibleModule(module='''
export async function renderFunction(options) {
    const status = document.createElement('div');
    status.style.background = 'red';
    status.textContent = 'No comms';
    options.node.appendChild(status);
        
    const commTargetName = options.data.data['application/vnd.notebook.json'].comm_target_name;
    const comms = await options.createComm(commTargetName);
    processMessages(comms);
    
    async function processMessages(comms) {
        let passed = false;
        for await (const message of comms.messages) {
            if (message.data.passed) {
                passed = true;
                comms.close();
            }
        }
        if (passed) {
            status.style.background = 'lime';
            status.textContent = 'passed';
        }
    }
}
''')
module.data['application/vnd.notebook.json'] = {
    'comm_target_name': 'my_comm_channel'
}
module

<__main__.ExtensibleModule object>